In [1]:
#Written by ChatGPT

import heapq
import math

class FibonacciHeapNode:
    def __init__(self, node, key):
        self.node = node
        self.key = key
        self.parent = None
        self.child = None
        self.left = self
        self.right = self
        self.degree = 0
        self.marked = False

class FibonacciHeap:
    def __init__(self):
        self.min_node = None
        self.nodes = {}

    def insert(self, node, key):
        new_node = FibonacciHeapNode(node, key)
        self.nodes[node] = new_node
        if not self.min_node or key < self.min_node.key:
            self.min_node = new_node

    def extract_min(self):
        if self.min_node:
            min_node = self.min_node
            if min_node.child:
                child = min_node.child
                while True:
                    next_child = child.right
                    child.left = min_node
                    child.right = min_node.right
                    min_node.right.left = child
                    min_node.right = child
                    if next_child == min_node.child:
                        break
                    child = next_child
            min_node.left.right = min_node.right
            min_node.right.left = min_node.left
            if min_node == min_node.right:
                self.min_node = None
            else:
                self.min_node = min_node.right
                self.consolidate()
            del self.nodes[min_node.node]
            return min_node.node, min_node.key
        return None, None

    def consolidate(self):
        max_degree = int(2 * (math.log(len(self.nodes)) / math.log(1 + math.sqrt(5))))
        degree_table = [None] * (max_degree + 1)
        current = self.min_node
        unprocessed_nodes = [current]
        while True:
            current = current.right
            if current == self.min_node:
                break
            unprocessed_nodes.append(current)
        for node in unprocessed_nodes:
            degree = node.degree
            while degree_table[degree]:
                other = degree_table[degree]
                if node.key > other.key:
                    node, other = other, node
                self.link(other, node)
                degree_table[degree] = None
                degree += 1
            degree_table[degree] = node
        for entry in degree_table:
            if entry and entry.key < self.min_node.key:
                self.min_node = entry

    def link(self, node1, node2):
        node1.right.left = node1.left
        node1.left.right = node1.right
        node1.parent = node2
        if not node2.child:
            node2.child = node1
            node1.right = node1
            node1.left = node1
        else:
            node1.left = node2.child
            node1.right = node2.child.right
            node2.child.right.left = node1
            node2.child.right = node1
        node2.degree += 1
        node1.marked = False

# def dijkstra(graph, start):
#     distances = {}
#     predecessors = {}
#     heap = FibonacciHeap()
    
#     for node in graph:
#         distances[node] = float('inf')
#     distances[start] = 0
#     heap.insert(start, 0)
#     while heap.min_node:
#         current_node, current_distance = heap.extract_min()
#         print("Current Node:", current_node, "Distance:", current_distance)
#         if current_distance > distances[current_node]:
#             continue
        
#         for neighbor, weight in graph[current_node].items():
#             #print('neighbor', neighbor, 'cur_node', current_node)
#             distance = current_distance + weight
#             #print(distances, neighbor)
#             if distance < distances[neighbor]:
#                 distances[neighbor] = distance
#                 predecessors[neighbor] = current_node
#                 heap.insert(neighbor, distance)
#     #print('Done with Dijkstra')
#     #print('dijkstra', distances)
#     return distances, predecessors

def dijkstra(graph, start):
    distances = {node: float('inf') for node in graph}
    predecessors = {node: None for node in graph}
    distances[start] = 0
    
    priority_queue = [(0, start)]
    
    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)
        
        if current_distance > distances[current_node]:
            continue
        #print(graph)
        for neighbor, weight in graph[current_node].items():
            distance = current_distance + weight
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                predecessors[neighbor] = current_node  # Store the predecessor
                heapq.heappush(priority_queue, (distance, neighbor))
    
    return distances, predecessors

def shortest_path(graph, start, end, predecessors):
    path = []
    while end:
        path.insert(0, end)
        end = predecessors.get(end, None)
    return path

# Example usage:
graph = {
    'A': {'B': 2, 'D': 8, 'G' : 6},
    'B': {'A': 2, 'D': 5, 'E': 6},
    'C': {'E': 9, 'F': 3},
    'D': {'A': 8, 'B': 5, 'E': 3, 'F': 2},
    'E': {'B': 6, 'C': 9, 'D': 3, 'F': 1, 'G': 4},
    'F': {'D': 2, 'E': 1, 'C': 3},
    'G': {'A': 6, 'E': 4}
}

start_node = 'A'
end_node = 'F'

shortest_distance, predecessors = dijkstra(graph, start_node)
shortest_path_to_D = shortest_path(graph, start_node, end_node, predecessors)

print(f"Shortest distance from {start_node} to {end_node}: {shortest_distance}")
print(f"Shortest path from {start_node} to {end_node}: {' -> '.join(shortest_path_to_D)}")

Shortest distance from A to F: {'A': 0, 'B': 2, 'C': 12, 'D': 7, 'E': 8, 'F': 9, 'G': 6}
Shortest path from A to F: A -> B -> D -> F


In [2]:
import heapq
#import heapq



In [3]:
predecessors

{'A': None, 'B': 'A', 'C': 'F', 'D': 'B', 'E': 'B', 'F': 'D', 'G': 'A'}

In [4]:
def create_directed_graph(graph, shortest_distance):
    '''Creates a directed graph as specified on page 19 or 245 of AN19.'''
    directed = {}
    for u in graph: #first vertex
        if u not in directed:
            directed[u] = {}
        for v in graph[u]: #second vertex
            directed[u][v] = graph[u][v] - (shortest_distance[v] - shortest_distance[u])
            #directed[v][u] = graph[v][u] - (shortest_distance[u] - shortest_distance[v])
    return directed

In [5]:
d = create_directed_graph(graph, shortest_distance)

In [6]:
d

{'A': {'B': 0, 'D': 1, 'G': 0},
 'B': {'A': 4, 'D': 0, 'E': 0},
 'C': {'E': 13, 'F': 6},
 'D': {'A': 15, 'B': 10, 'E': 2, 'F': 0},
 'E': {'B': 12, 'C': 5, 'D': 4, 'F': 0, 'G': 6},
 'F': {'D': 4, 'E': 2, 'C': 0},
 'G': {'A': 12, 'E': 2}}

In [7]:
dijkstra(d, 'E')

({'A': 16, 'B': 12, 'C': 0, 'D': 4, 'E': 0, 'F': 0, 'G': 6},
 {'A': 'B', 'B': 'E', 'C': 'F', 'D': 'E', 'E': None, 'F': 'E', 'G': 'E'})

In [8]:
dijkstra(graph, 'A')

({'A': 0, 'B': 2, 'C': 12, 'D': 7, 'E': 8, 'F': 9, 'G': 6},
 {'A': None, 'B': 'A', 'C': 'F', 'D': 'B', 'E': 'B', 'F': 'D', 'G': 'A'})

In [9]:
def is_undirected(graph):
    # Create a set to store checked edges
    checked_edges = set()

    # Iterate through each node in the graph
    for node, neighbors in graph.items():
        for neighbor, length in neighbors.items():
            # Check if the reverse edge exists and has the same length
            if (neighbor, node) in checked_edges:
                if graph[neighbor][node] != length:
                    print(f'neigbhor = {neighbor} and node = {node}')
                    return False
            else:
                checked_edges.add((node, neighbor))

    return True

In [10]:
def is_connected(graph):
    if not graph:
        # An empty graph is considered connected
        return True

    def dfs(node, visited):
        visited[node] = True
        for neighbor in graph[node]:
            if not visited[neighbor]:
                dfs(neighbor, visited)

    # Initialize a dictionary to keep track of visited nodes
    visited = {node: False for node in graph}

    # Choose any node as the starting point
    start_node = next(iter(graph))
    dfs(start_node, visited)

    # Check if all nodes have been visited
    return all(visited[node] for node in graph)

# Example usage:
graph = {
    'A': {'B': 2, 'D': 8, 'G' : 6},
    'B': {'A': 2, 'D': 5, 'E': 6},
    'C': {'E': 9, 'F': 3},
    'D': {'A': 8, 'B': 5, 'E': 3, 'F': 2},
    'E': {'B': 6, 'C': 9, 'D': 3, 'F': 1, 'G': 4},
    'F': {'D': 2, 'E': 1, 'C': 3},
    'G': {'A': 6, 'E': 4}
}

connected = is_connected(graph)
if connected:
    print("The graph is connected.")
else:
    print("The graph is not connected.")

The graph is connected.


In [11]:
# def create_petal(graph, x, t, r, directed = None): #Edit this to take in the directed graph as a parameter Y that is computed separately
#     assert x in graph
#     assert t in graph
#     shortest_distance, predecessors = dijkstra(graph, x)
#     if directed == None:
#         directed = create_directed_graph(graph, shortest_distance)
    
#     shortest_path_to_t = shortest_path(graph, x, t, predecessors)
#     if x != t and len(shortest_path_to_t) == 1:
#         print(x, t, shortest_distance[t])
#     shortest_distance_directed, predecessors_directed = dijkstra(directed, t)
#     t_dist = shortest_distance_directed[t]
#     r_prime = x
#     shortest_path_rprime = []
#     r_prime_idx = -1
#     for i in range(len(shortest_path_to_t)):
#         v = shortest_path_to_t[i]
#         d = t_dist - shortest_distance[v]

#         if d < r:
#             r_prime = v
#             r_prime_idx = i
#             break
    
#     for i in range(r_prime_idx, len(shortest_path_to_t)):
#         shortest_path_rprime.append(shortest_path_to_t[i])
    
#     petal = {}
#     for v in shortest_distance_directed:
#         if shortest_distance_directed[v] < r/2:
#             petal[v] = {}
#     for v in shortest_path_rprime:
#         petal[v] = {}
        
#     for u in petal:
#         for v in graph[u]:
#             if v in petal:
#                 petal[u][v] = graph[u][v]

#     if not is_connected(petal):
#         print(petal)
#         raise ValueError(f'Petal = {petal} not connected')
#     if len(petal) == 1:
#         r_prime = list(petal.keys())[0]
#     return petal, r_prime, shortest_path_to_t

In [12]:
def create_petal(graph, x, t, r): #Edit this to take in the directed graph as a parameter Y that is computed separately
    assert x in graph
    assert t in graph
    shortest_distance, predecessors = dijkstra(graph, x)
    directed = create_directed_graph(graph, shortest_distance)
    
    shortest_path_to_t = shortest_path(graph, x, t, predecessors)
    if x != t and len(shortest_path_to_t) == 1:
        print(x, t, shortest_distance[t])
    shortest_distance_directed, predecessors_directed = dijkstra(directed, t)

    t_dist = shortest_distance[t]
    assert t_dist != float('inf')
    r_prime = t
    r_prime_idx = -1
    #shortest_path_rprime = []
    
    for v in shortest_path_to_t:
        #shortest_path_rprime.append(v)
        r_prime_idx += 1
        if t_dist - shortest_distance[v] < r:
            r_prime = v
            break
    shortest_path_rprime = shortest_path_to_t[r_prime_idx:]
    
    petal = {}
    for v in shortest_distance_directed:
        if shortest_distance_directed[v] < r/2:
            petal[v] = {}
    for v in shortest_path_rprime:
        petal[v] = {}
        
    for u in petal:
        for v in graph[u]:
            if v in petal:
                petal[u][v] = graph[u][v]

    if not is_connected(petal):
        print(petal)
        raise ValueError(f'Petal = {petal} not connected')
    if len(petal) == 1:
        r_prime = list(petal.keys())[0]

    return petal, r_prime, shortest_path_to_t 

In [13]:
#graph = {'0': {'1': 37.0}, '1': {'0': 37.0, '6': 92.0}, '2': {'3': 5.0}, '3': {'2': 5.0, '4': 30.0}, '4': {'3': 30.0, '6': 20.0}, '5': {'6': 4.0}, '6': {'5': 4.0, '4': 20.0, '1': 92.0}}

In [14]:
#hierarchical_petal_decomposition(graph, '1', '5')

In [15]:
create_petal(graph, 'A', 'C', 12 - 5*12/8)

{'A': {'B': 0, 'D': 1, 'G': 0}, 'B': {'A': 4, 'D': 0, 'E': 0}, 'C': {'E': 13, 'F': 6}, 'D': {'A': 15, 'B': 10, 'E': 2, 'F': 0}, 'E': {'B': 12, 'C': 5, 'D': 4, 'F': 0, 'G': 6}, 'F': {'D': 4, 'E': 2, 'C': 0}, 'G': {'A': 12, 'E': 2}}


({'C': {'F': 3}, 'F': {'C': 3}}, 'F', ['A', 'B', 'D', 'F', 'C'])

In [16]:
graph

{'A': {'B': 2, 'D': 8, 'G': 6},
 'B': {'A': 2, 'D': 5, 'E': 6},
 'C': {'E': 9, 'F': 3},
 'D': {'A': 8, 'B': 5, 'E': 3, 'F': 2},
 'E': {'B': 6, 'C': 9, 'D': 3, 'F': 1, 'G': 4},
 'F': {'D': 2, 'E': 1, 'C': 3},
 'G': {'A': 6, 'E': 4}}

In [17]:
def maintain_graph_difference(G, H):
    result = {}

    for u, neighbors in G.items():
        if u not in H:
            result[u] = {}
        
        for v in neighbors:
            if u not in H and v not in H:
                if u not in result:
                    result[u] = {}
                result[u][v] = G[u][v]

    return result

In [18]:
graph

{'A': {'B': 2, 'D': 8, 'G': 6},
 'B': {'A': 2, 'D': 5, 'E': 6},
 'C': {'E': 9, 'F': 3},
 'D': {'A': 8, 'B': 5, 'E': 3, 'F': 2},
 'E': {'B': 6, 'C': 9, 'D': 3, 'F': 1, 'G': 4},
 'F': {'D': 2, 'E': 1, 'C': 3},
 'G': {'A': 6, 'E': 4}}

In [19]:
maintain_graph_difference(graph, ['A', 'B'])

{'C': {'E': 9, 'F': 3},
 'D': {'E': 3, 'F': 2},
 'E': {'C': 9, 'D': 3, 'F': 1, 'G': 4},
 'F': {'D': 2, 'E': 1, 'C': 3},
 'G': {'E': 4}}

In [105]:
def petal_decomposition(graph, x, t):
    assert x in graph
    if t not in graph:
        raise ValueError(f't = {t} not in graph vertices = {list(graph.keys())}')
    shortest_distance, predecessors = dijkstra(graph, x)
    r = max(shortest_distance.values())
    #print(f'radius = {r}')
    #print('d', shortest_distance)
    Ys = [graph]
    Xs = []
    ys = []
    xs = [x]
    ts = []
    j = 1
    if shortest_distance[t] > 5*r/8:
        
        X_1, x_1, path = create_petal(graph, x, t, shortest_distance[t] - 5*r/8)
        print('special petal', X_1, x_1, t, path)
        #print('HI', X_1, x_1, t)
        #Ys.append({key : value for key, value in Ys[0].items() if key not in X_1})
        Ys.append(maintain_graph_difference(Ys[0], X_1))
        Xs.append(X_1)
        xs.append(x_1)
        
        y1 = x
        min_distance = r
        #if x_1 != t:
        #path = shortest_path(graph, x, t, predecessors)
        #print('path', path)
        for i in range(len(path)):
            if path[i] == x_1:
                y1 = path[i-1]
                break

        j = 2
        ts = [y1, t]
        ys = [y1]
        if y1 in X_1 and x_1 != t:
            raise ValueError(f'y1 = {y1} should not be in X1 = {X_1}')
        
    else:
        ts = [t] 
    while True:
        shortest_distance, predecessors = dijkstra(Ys[-1], x)
        #print(graph)
        ball = {key : value for key, value in shortest_distance.items() if shortest_distance[key] < 3*r/4}
        leftover = [v for v in Ys[-1] if v not in ball and shortest_distance[v] != float('inf')] #maintain_graph_difference(Ys[-1], ball)
        if len(leftover) == 0:
            break
        else:
            #print('leftover', leftover, '\n')
            ts.append(leftover[0])
            #print(leftover[0])
            #print(x, r/8, Ys[-1])
            #print('dist to tj', shortest_distance[leftover[0]])
            assert x in Ys[-1]
            assert ts[-1] in Ys[-1]
            X_j, x_j, path = create_petal(Ys[-1], x, ts[-1], r/8)
            assert x_j in Ys[-1]
            #print(f'graph = {graph}, x = {x}, t_j = {ts[-1]}, r = {r/8}, Ys[-1] = {Ys[-1]}')
            #print(f'X_j = {X_j}, graph = {graph}, x = {x}, t_j = {ts[-1]}, r = {r/8}, Ys[-1] = {Ys[-1]}')
            if len(path) == 1:
                print('path of length 1:', x, x_j, ts[-1])
            if x_j not in X_j:
                print(f'x_j = {x_j} not in X_j = {X_j}')
                print(f'graph = {graph}, x = {x}, t_j = {ts[-1]}, r = {r/8}, Ys[-1] = {Ys[-1]}')
                print(f'Xs = {Xs}, xs = {xs}, ys = {ys}, ts = {ts}, Ys = {Ys}')
                raise ValueError
            assert ts[-1] in X_j
            Xs.append(X_j)
            xs.append(x_j)
            Ys.append(maintain_graph_difference(Ys[-1], X_j))
            xj_idx = -1
            if x_j not in path:
                print(f'x_j {x_j} not found in path {path} of length {shortest_distance[ts[-1]]} from {x} to {ts[-1]}')
            for i in range(len(path)):
                if path[i] == x_j:
                    xj_idx = i
                    #print(i)
                # else:
                #     print(f'x_j {x_j} not found in path {path}')
                if xj_idx > -1 and i < len(path) - 1:
                    #print('g', graph, path[i-1], path[i], path, i)
                    graph[path[i]][path[i+1]] /= 2
                    graph[path[i+1]][path[i]] /= 2
                    #print('hi')
                    #shortest_distance[path[i]] = (shortest_distance[path[i]] + shortest_distance[path[xj_idx]])/2
            if x_j == x:
                print('hello')
                ys.append(path[0])
            else:
                ys.append(path[xj_idx - 1])
                #print('path', path, 'x_j', path[xj_idx], 'y_j', path[xj_idx - 1])
            j += 1
            if ys[-1] == xs[-1]:
                print('x = y', path)
                #raise ValueError('x = y')
            if ys[-1] not in graph[xs[-1]]:
                raise ValueError(f'{xs[-1]}, {ys[-1]} not an edge in graph')
            #print('ys', ys)
        #print('Completed while')
    s = j - 1
    #print(s, len(Ys))
    X0 = Ys[s]
    X0_keys = list(X0.keys())
    # for key in X0_keys:
    #     if key not in graph[x]:
    #         raise ValueError(f'Vertex {key} not in adjacent vertices of x_0: {graph[x]}')
    if not is_connected(X0):
        for v in X0:
            if len(X0[v]) == 0:
                print(f'Isolated vertex {v}. Edges connected to v: {graph[v]}')
                for X in Xs:
                    if v in X:
                        print(f'Isolated vertex {v} in cluster {X}')
        raise ValueError(f'X0 = {X0} not connected. Edges connected to x_0 = {x}: {graph[x]}')
    Xs = [X0] + Xs
    return Xs, ys, xs, ts
        

In [21]:
#create_petal(graph, '4', '2', 15.25)

In [22]:
#petal_decomposition(graph, '0', '2')

In [23]:
def make_undirected(graph):
    undirected_graph = {}
    for u in graph:
        for v, length in graph[u].items():
            # Add edge from u to v
            if u not in undirected_graph:
                undirected_graph[u] = {}
            undirected_graph[u][v] = length
            
            # Add edge from v to u
            if v not in undirected_graph:
                undirected_graph[v] = {}
            undirected_graph[v][u] = length
    
    return undirected_graph

In [24]:
def is_tree(graph):
    num_vertices = len(graph)
    num_edges = sum(len(adj_list) for adj_list in graph.values()) // 2  # Divide by 2 to account for double-counting edges

    return num_edges == num_vertices - 1

# Example usage

if is_tree(graph):
    print("The graph is a tree.")
else:
    print("The graph is not a tree.")


The graph is not a tree.


In [25]:
def hierarchical_petal_decomposition(graph, x_0, t):
    
    if len(graph) == 1:
        return graph
    if is_tree(graph):
        return graph
    Xs, ys, xs, ts = petal_decomposition(graph, x_0, t)

    Ts = []
    for j in range(len(Xs)):
        if xs[j] not in Xs[j]:
            print(f'xj = {xs[j]} not in {Xs[j]}')
        if ts[j] not in Xs[j]:
            print(f'tj = {xs[j]} not in {Xs[j]}')
        Ts.append(hierarchical_petal_decomposition(Xs[j], xs[j], ts[j]))

    T = {}
    for tree in Ts:
        if not is_undirected(tree):
            print(tree)
            raise ValueError('tree is not undirected')
        for u in tree:
            T[u] = tree[u]
    #print(xs, ys)
    if len(xs) > 1:
        for i in range(1, len(xs)):
            x = xs[i]
            y = ys[i-1]
            T[x][y] = graph[x][y]
            T[y][x] = graph[y][x]
    
    return make_undirected(T)
        

In [26]:
tree = hierarchical_petal_decomposition(graph, 'A', 'A')

In [27]:
tree

{'A': {'B': 2, 'G': 6},
 'B': {'A': 2, 'D': 5, 'E': 6},
 'G': {'A': 6},
 'D': {'B': 5, 'F': 2},
 'E': {'B': 6},
 'F': {'D': 2, 'C': 3},
 'C': {'F': 3}}

In [28]:
import math

def round_edge_lengths_to_power_of_2(graph):
    rounded_graph = {}

    for node, edges in graph.items():
        rounded_edges = {}

        for neighbor, length in edges.items():
            rounded_length = 2 ** int(math.log2(length))
            rounded_edges[neighbor] = rounded_length

        rounded_graph[node] = rounded_edges

    return rounded_graph

rounded_graph = round_edge_lengths_to_power_of_2(graph)
print(rounded_graph)

{'A': {'B': 2, 'D': 8, 'G': 4}, 'B': {'A': 2, 'D': 4, 'E': 4}, 'C': {'E': 8, 'F': 2}, 'D': {'A': 8, 'B': 4, 'E': 2, 'F': 2}, 'E': {'B': 4, 'C': 8, 'D': 2, 'F': 1, 'G': 4}, 'F': {'D': 2, 'E': 1, 'C': 2}, 'G': {'A': 4, 'E': 4}}


### Tree

In [29]:
import numpy as np
from tqdm import tqdm
def calculate_stretch(graph, x_0, t):
    rounded_graph = round_edge_lengths_to_power_of_2(graph)
    T = hierarchical_petal_decomposition(graph, x_0, t)
    assert is_undirected(T)
    assert is_tree(T)
    #T = make_undirected(T)
    #print(T)
    if len(T) != len(graph):
        raise ValueError(f'Not a spanning tree. T has {len(t)} vertices while the original graph has {len(graph)} vertices')
    m = len(graph)
    n = 0
    #t_length = 0
    #graph_length = 0
    stretch = 0
    graph_keys_list = list(graph.keys())
    for i in tqdm(range(len(graph_keys_list))):
        u = graph_keys_list[i]
        for v in graph[u]:
            assert u in T and v in T
            n += 1
            distances, _ = dijkstra(T, u)
            if distances[v] == float('inf'):
                print('INF', u, v, T)
                print(is_undirected(T))
                print(is_connected(T))
                raise ValueError('Distance should be finite')
            stretch += distances[v]/graph[u][v]
    print(f'fraction of edges: {(n/2)/(m*(m-1)/2) }')
    n /= 2
    bound = m * np.log(n) * np.log(np.log(n))
    #print(f'stretch = {stretch}')
    return stretch/bound

In [30]:
import random

def generate_graph(m, p):
    if m <= 1:
        # Special case: a single vertex graph is always connected.
        return {'0': {}}

    # Create a list of vertices as strings from '0' to 'm-1'.
    vertices = [str(i) for i in range(m)]

    # Create a dictionary to represent the graph with edge lengths.
    graph = {v: {} for v in vertices}

    # Create a list of edges with associated probabilities.
    edges = []

    for i in range(m):
        for j in range(i + 1, m):
            if random.random() < p:
                edges.append((vertices[i], vertices[j]))

    # Create a minimum spanning tree using Prim's algorithm.
    random.shuffle(vertices)
    visited = set([vertices[0]])
    while len(visited) < m:
        current_vertex = random.choice(list(visited))
        remaining_vertices = [v for v in vertices if v not in visited]
        if not remaining_vertices:
            break
        next_vertex = random.choice(remaining_vertices)
        visited.add(next_vertex)
        edge = (current_vertex, next_vertex)
        edges.append(edge)

    # Shuffle the list of edges to randomize the order.
    random.shuffle(edges)

    # Add the edges to the graph.
    for (start_vertex, end_vertex) in edges:
        edge_length = float(random.randint(1.0, 100.0))  # Modify this range as needed
        graph[start_vertex][end_vertex] = edge_length
        graph[end_vertex][start_vertex] = edge_length

    return graph

# # Example usage:
# m = 5
# p = 0.5  # Probability of an edge between any pair of vertices
# connected_graph = generate_connected_graph_with_probability(m, p)
# print(connected_graph)

In [31]:
def furthest_point(graph, x_0):
    distances, _ = dijkstra(graph, x_0)
    #print(distances)
    max_dist = max(distances.values())
    for v in distances:
        if distances[v] == max_dist:
            return v

In [100]:
import random

# Create a graph represented as a dictionary

def test_lsst(m, p):

#     graph = {}
#     vertices = [str(i) for i in range(m)]

#     # Create edges with random lengths
#     for i in range(m):
#         for j in range(i + 1, m):
#             if np.random.uniform() < p:
#                 edge_length = np.random.randint(1, 100)  # Random edge length between 1 and 10
#                 if vertices[i] not in graph:
#                     graph[vertices[i]] = {}
#                 if vertices[j] not in graph:
#                     graph[vertices[j]] = {}
#                 graph[vertices[i]][vertices[j]] = edge_length
#                 graph[vertices[j]][vertices[i]] = edge_length  # Undirected, so add the reverse edge
#             else:
#                 print('oops')

    graph = generate_graph(m, p)
    assert is_connected(graph)
            
    v1 = str(np.random.randint(0, m))
    
    #print(graph, v1)
    return calculate_stretch(graph, v1, v1)

# Example: Accessing the length of the edge between vertex '0' and '1'
#edge_length_01 = graph['0']['1']

In [106]:
import time
n = 500
for i in range(n):
    for num_vertices in [200]: #, 1000, 5000]:
        print(f'Starting trial {i + 1} of {n}')
        p = np.log(num_vertices)/num_vertices
        print(test_lsst(num_vertices, 0.1))#np.log(1000)/1000))
            
            
        #print(f'LSST on graph with {num_vertices} vertices')
        # start = time.time()
        # print(test_lsst(num_vertices, 1))
        # end = time.time()
        #print(f'Took {end - start} seconds')

Starting trial 1 of 500


100%|██████████| 200/200 [00:00<00:00, 258.64it/s]


fraction of edges: 0.10768844221105528
1.9606583810537612
Starting trial 2 of 500


100%|██████████| 200/200 [00:00<00:00, 254.53it/s]


fraction of edges: 0.10974874371859296
2.475494677731936
Starting trial 3 of 500


100%|██████████| 200/200 [00:00<00:00, 250.81it/s]


fraction of edges: 0.1093467336683417
1.9964089071047717
Starting trial 4 of 500


100%|██████████| 200/200 [00:00<00:00, 248.20it/s]


fraction of edges: 0.11060301507537688
2.0021754845943214
Starting trial 5 of 500


ValueError: X0 = {'31': {'188': 2.0}, '40': {'77': 5.0}, '77': {'78': 8.0, '133': 1.0, '40': 5.0}, '78': {'77': 8.0}, '133': {'77': 1.0}, '188': {'31': 2.0}} not connected. Edges connected to x_0 = 40: {'132': 19.0, '116': 18.0, '77': 5.0}

### Debugging

In [79]:
graph = {'0': {'4': 59.0}, '1': {'2': 65.0, '3': 24.0, '4': 97.0}, '2': {'1': 65.0, '3': 1.0}, '3': {'1': 24.0, '2': 1.0}, '4': {'0': 59.0, '1': 97.0}}

In [80]:
graph

{'0': {'4': 59.0},
 '1': {'2': 65.0, '3': 24.0, '4': 97.0},
 '2': {'1': 65.0, '3': 1.0},
 '3': {'1': 24.0, '2': 1.0},
 '4': {'0': 59.0, '1': 97.0}}

['1', '4']


({'1': {}}, '1', ['4', '1'])

In [80]:
len({})

0

In [86]:
g = {'1': {'16': 2.0, '84': 77.0}, '6': {'88': 5.0, '87': 5.0, '33': 89.0}, '16': {'1': 2.0, '92': 20.0, '91': 17.0, '87': 8.0}, '33': {'48': 29.0, '6': 89.0, '45': 7.0, '87': 13.0}, '34': {'89': 82.0, '45': 71.0, '48': 86.0, '88': 2.0}, '45': {'84': 8.0, '89': 11.0, '34': 71.0, '33': 7.0}, '48': {'33': 29.0, '92': 29.0, '34': 86.0, '88': 47.0}, '49': {'92': 77.0, '84': 2.0, '87': 22.0, '55': 84.0, '88': 58.0}, '55': {'91': 17.0, '92': 2.0, '49': 84.0}, '84': {'45': 8.0, '49': 2.0, '1': 77.0}, '85': {}, '87': {'6': 5.0, '49': 22.0, '16': 8.0, '33': 13.0}, '88': {'6': 5.0, '34': 2.0, '49': 58.0, '48': 47.0}, '89': {'34': 82.0, '45': 11.0, '92': 45.0}, '91': {'55': 17.0, '16': 17.0, '96': 3.0}, '92': {'49': 77.0, '55': 2.0, '48': 29.0, '16': 20.0, '89': 45.0}, '96': {'91': 3.0}}

In [87]:
is_connected(g)

False

In [88]:
del g['85']

In [89]:
is_connected(g)

True

In [113]:
for i in tqdm(range(1, 200)):
    for j in range(1, 200):
        for k in range(1, 200):
            for l in range(1, 200):
                if i**4 == j**4 + k**4 + l**4 and i != j and i !=k and i != l:
                    print(i, j, k, l)

  1%|          | 2/199 [00:12<19:47,  6.03s/it]


KeyboardInterrupt: 

In [114]:
20615673**4 == 2682440**4 + 15365639**4 + 18796760**4

True

In [115]:
print(20615673**4)

180630077292169281088848499041


In [117]:
print(2682440**4 + 15365639**4 + 18796760**4)

180630077292169281088848499041
